In [1]:
#Use full browser width for ipynb coding.
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

/var/folders/k5/ng4bvm413sq33q7jvcpyyt840000gn/T/ipykernel_76562/1841757209.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Project setup

## Imports

In [2]:
import lxml

In [3]:
import sys

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import re

import psycopg2
from psycopg2 import OperationalError
from io import BytesIO
from PIL import Image
from IPython.display import display

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains

# Initialize the Chrome WebDriver with custom options
chrome_options = webdriver.ChromeOptions()

# Add user agent to mimic a real browser (optional but helps)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.9999.999 Safari/537.36")

#!pip install selenium
#!pip install webdriver-manager

## Function definition

In [4]:
# Function to download an image from a URL and convert it to binary data
def download_image(url):
    #ici ca marche pas à la lecture après écriture. Pourquoi?
    headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        image = response.content
        return image
    else:
        return None

In [5]:
def create_database(db_name, db_user, db_password, db_host, db_port):
    try:
        # Connect to the default "postgres" database to create a new database
        conn = psycopg2.connect(
            database="postgres",
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Create the new database
        cursor.execute(f"CREATE DATABASE {db_name}")
        
        print(f"Database '{db_name}' created successfully!")

    except OperationalError as e:
        print(f"Error: {e}")
    finally:
        if conn:
            conn.close()

In [6]:
# Initialize the WebDriver and login

def driver_init(login_url):
    driver = webdriver.Safari()
    driver.set_window_size(1920, 1480)

    # Navigate to the URL
    URL = login_url
    driver.start_client()
    driver.get(URL)

    # Wait for JavaScript to load (you can customize the wait time)
    driver.implicitly_wait(1)
    
    driver.get(login_url) #Access website
    driver.implicitly_wait(2)

    try:
        # Find the login form elements and fill in your login credentials
        username_input = driver.find_element(by="id", value="username")
        password_input = driver.find_element(by="id", value="password")

        # Enter your login credentials (replace with your own)
        username_input.send_keys("trololo")
        password_input.send_keys("trololo123AZE")
        driver.implicitly_wait(2)

        # Submit the login form
        login_button = driver.find_element(by="id", value="loginButton")
        login_button.submit() #Click login button. ie submit login credentials

        # You may need to handle CAPTCHA here if it appears
        # You can use CAPTCHA solving services (e.g., 2Captcha) or manual intervention
    except Exception as e:
        print("An error occurred:", e)
        
    # Wait for some time (adjust as needed) to allow for successful login
    driver.implicitly_wait(2)
        
    return driver # After login, we can perform additional actions as needed

In [7]:
#Inherited from v1
def query_db(query, conn_params):
    """ create tables in the PostgreSQL database"""
    
    conn = None
    try:
        # read the connection parameters
        #params = config() 
        # connect to the PostgreSQL server
        #conn = psycopg2.connect(**params)
        conn = psycopg2.connect(conn_params)
        cur = conn.cursor()
        # create table one by one
        cur.execute(query)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

# Data Retrieval

In [8]:
# Login Page Url
url = "https://jewelry.ha.com/c/login.zx?source=nav"
driver_ = driver_init(login_url = url)

In [10]:
# Designer marks URL
designermarks_href="https://jewelry.HA.com/ref/designer-marks.zx?ic=Tab-Resources-DesignerMarks-121516-interior"

In [11]:
# Navigate to designer mark URL
driver_.get(designermarks_href)
driver_.implicitly_wait(2)

In [12]:
# Get HTML Code from current page. Allows us to get all designer names
html_code = driver_.page_source
driver_.implicitly_wait(2)

In [13]:
# Let's now use selenium to parse our HTML code

In [14]:
# Write data to file
with open("parsed_designermarks_html.html", "w", encoding="utf-8") as file:
            file.write(html_code)

In [15]:
# Images are not loaded in this HTML code since they load on scroll.

In [13]:
# Let's download those images
soup = bs(html_code, 'lxml')
print(soup.img)

#Extract desiner names and picture url
for i in soup.find_all(class_ = 'mark-link lazyload ha-tooltip preview'):
    print(i.attrs["alt"].replace(" maker's mark",""))
    print(i.attrs["data-image1"]) #For future me : no data-image2 exists. each image entry is a specific lazyload class entry, this loop covers it.
    #break #in order to test only one entry. Can be removed to see all entries

<img alt="Heritage Auctions logo" src="https://www.heritagestatic.com/c/i/ui/skin/logo.png"/>
Ambrosi
https://www.heritagestatic.com/c/i/articles/designers/full-size/ambrosi1.jpg
Andreoli
https://www.heritagestatic.com/c/i/articles/designers/full-size/andreoli1.jpg
Andreoli
https://www.heritagestatic.com/c/i/articles/designers/full-size/andreoli4.jpg
Andrew Clunn
https://www.heritagestatic.com/c/i/articles/designers/full-size/andrew-clunn3.jpg
Andrew Grima
https://www.heritagestatic.com/c/i/articles/designers/full-size/andrew-grima2.jpg
Andrew Grima
https://www.heritagestatic.com/c/i/articles/designers/full-size/andrew-grima3.jpg
Angela Cummings
https://www.heritagestatic.com/c/i/articles/designers/full-size/angela-cummings5.jpg
Angela Cummings
https://www.heritagestatic.com/c/i/articles/designers/full-size/angela-cummings20.jpg
Angela Cummings for Tiffany & Co.
https://www.heritagestatic.com/c/i/articles/designers/full-size/angela-cummings21.jpg
Angela Cummings for Tiffany & Co.
https

# Data Engineering/Architecture part

In [16]:
# Creates a list of dictionnaries. Each dict contains name, url and image_url (later replaced by image binary data).
data=[]
for i in soup.find_all(class_ = 'mark-link lazyload ha-tooltip preview'):
    entry={}
    entry["name"]= i.attrs["alt"].replace(" maker's mark","")
    entry["url"] = i.attrs["data-image1"]
    entry["image_url"]= i.attrs["data-image1"]
    data.append(entry)

In [17]:
# Check for a specific entry
for i in data:
    #print(i)
    if i.get("name") == "Louis Fiessler":
        print(i)

{'name': 'Louis Fiessler', 'url': 'https://www.heritagestatic.com/c/i/articles/designers/full-size/louis-fiessler.jpg', 'image_url': 'https://www.heritagestatic.com/c/i/articles/designers/full-size/louis-fiessler.jpg'}


## Download & Store Images for the MakersMarks Page

In [18]:
# This is already done in the v1 script, only need to copy past

## Download & Store Infos of each Designer Page 

### Get links to each designer past sales page

In [15]:
# Get links for all designers

for i in soup.find_all(class_ = 'mark-links'): #Get three links
    all_links=i.find_all('a') #Store them
    print(all_links[1]) # Only extract the second entry, ie the "value" link
    value_link=all_links[1]['href'] #Extract href text
    break

<a href="https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&amp;search=archives&amp;txtSearch=%22a.jaffe%22&amp;ic22=JewelryMM-Value-121119" target="NEW">Value</a>


In [16]:
# Navigate to a designer's listings through the "value" link

In [17]:
value_link

'https://jewelry.HA.com/c/search.zx?header=Estate+Jewelry&search=archives&txtSearch=%22a.jaffe%22&ic22=JewelryMM-Value-121119'

In [22]:
# Navigate to the value link, ie all listed items for the given designer
driver_.get(value_link)
driver_.implicitly_wait(2)

### Get all items links in past sales page

In [23]:
# Get the page source
designer_past_items_html_code = driver_.page_source

# Create a BeautifulSoup object from the page source
designer_past_items_soup = bs(designer_past_items_html_code, 'html.parser')

In [24]:
# Extract all past sales links
for i in designer_past_items_soup.find_all(class_ = 'item-title'): #Get three links
    print(i.get("href"))
    break

https://jewelry.ha.com/itm/estate-jewelry/rings/diamond-platinum-ring-a-jaffe/a/5427-55366.s?ic4=ListView-ShortDescription-071515


### Now let's take the first link and extract all useful info

In [25]:
one_past_designer_item_link= "https://jewelry.ha.com/itm/estate-jewelry/rings/diamond-platinum-ring-a-jaffe/a/5427-55366.s?ic4=ListView-ShortDescription-071515"


In [26]:
# Navigate to the value link, ie all listed items for the given designer
driver_.get(one_past_designer_item_link)
driver_.implicitly_wait(2)

In [ ]:
# Get the page source
one_past_designer_item_html_code = driver_.page_source

# Create a BeautifulSoup object from the page source
one_past_designer_item_soup = bs(one_past_designer_item_html_code, 'html.parser')

#### Text data

In [ ]:
#Sell date
one_past_designer_item_soup

In [ ]:
# Description
description=one_past_designer_item_soup.find(itemprop="description")
description.text

In [ ]:
# Condition Report
condition_report=one_past_designer_item_soup.find(id = 'condition-report')
condition_report.text

In [ ]:
# Sell date
sell_date=one_past_designer_item_soup.find(class_="section-headline").text.replace('Sold on ', '').replace(' for:', '')
sell_date

In [ ]:
# Auction number

In [ ]:
# Price
selling_price_raw=one_past_designer_item_soup.find(class_="opening-bid")
final_price=selling_price_raw.text.split('$')[1].strip().replace(',', '') #Only extract number after $character
final_price

In [ ]:
# Lot Number 
lot_number=one_past_designer_item_soup.find(class_="lot-picker").text
lot_number

# Search for the pattern in the input string
pattern = r'#(\d+)'  #find number after '#' symbol
match = re.search(pattern, lot_number)

# Extract the value if a match is found
if match:
    value_after_hash = match.group(1)
    print(value_after_hash)
else:
    print("No match found in the string.")

In [ ]:
# Images
images_links=[]
for i in one_past_designer_item_soup.find_all(itemprop="image"):
    if i["content"] not in images_links: #Check for duplicates
        images_links.append(i["content"]) # append to list
images_links

In [ ]:
# Let's create the function which roams throught the website and retrieves all the data.

#### Database Creation

In [40]:
 # Database connection parameters
db_name = "makersmark_db"
db_user = "postgres"
db_password = "makersmark"
db_host = "localhost"  # Change to your PostgreSQL server host
db_port = "5432"       # Change to your PostgreSQL server port

create_database(db_name, db_user, db_password, db_host, db_port)

Database 'makersmark_db' created successfully!


In [43]:
server_name="makersmark_v2_server"
# I think it's time to put some code in separate files and start using jupyterlab or vscode
db_creation_query= """CREATE TABLE designers (
    designer_id SERIAL PRIMARY KEY,
    designer_name VARCHAR(255) NOT NULL
);

CREATE TABLE items (
    item_id SERIAL PRIMARY KEY,
    designer_id INT REFERENCES designers(designer_id) ON DELETE CASCADE,
    item_description TEXT,
    item_condition_report TEXT,
    sell_price NUMERIC(12, 2) CHECK (sell_price >= 0),
    sell_date DATE,
    created_at TIMESTAMP DEFAULT NOW(),
    updated_at TIMESTAMP DEFAULT NOW(),
    CONSTRAINT valid_sell_date CHECK (sell_date <= CURRENT_DATE)
);

CREATE TABLE item_images (
    image_id SERIAL PRIMARY KEY,
    image_url VARCHAR(255),
    item_id INT REFERENCES items(item_id) ON DELETE CASCADE,
    designer_id INT REFERENCES designers(designer_id) ON DELETE CASCADE,
    binary_item_image_data BYTEA,
    created_at TIMESTAMP DEFAULT NOW(),
    updated_at TIMESTAMP DEFAULT NOW()
);

-- Indexes for improved query performance
CREATE INDEX idx_designer_name ON designers(designer_name);
CREATE INDEX idx_sell_date ON items(sell_date);
CREATE INDEX idx_item_url ON item_images(image_url);

-- Enforce unique designer names
ALTER TABLE designers ADD CONSTRAINT unique_designer_name UNIQUE (designer_name);

-- Audit Trail (track who created and updated records)
CREATE OR REPLACE FUNCTION update_timestamp() RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER update_items_timestamp
    BEFORE UPDATE ON items
    FOR EACH ROW
    EXECUTE FUNCTION update_timestamp();

CREATE TRIGGER update_item_images_timestamp
    BEFORE UPDATE ON item_images
    FOR EACH ROW
    EXECUTE FUNCTION update_timestamp();
"""

In [44]:
connection_params = "dbname=makersmark_db user=postgres password=makersmark"
query_db(db_creation_query, connection_params)

In [57]:
test_query_if_working='''SELECT * FROM designers;'''
results=query_db(test_query_if_working, connection_params)

In [58]:
print(results)

None


In [ ]:
# Now, how to store this shit? 
Whats my db schema?
Do i store a copy of the images locally. Could be fun? But could also take up a lot of space
What interval do i set for roaming through the website, in order not to get caught?

# Data Science part

In [ ]:
Data Augmentation : https://towardsdatascience.com/top-python-libraries-for-image-augmentation-in-computer-vision-2566bed0533e

# Prod Part : Create an iOS App?